# Free-for-all practice file, not included in final submission

In [1]:
import math

from bloqade import qasm2
from kirin.dialects import ilist

In [3]:
def ghz_linear(n: int):
    n_qubits = int(2**n)

    @qasm2.extended
    def ghz_linear_program():

        qreg = qasm2.qreg(n_qubits)
        # Apply a Hadamard on the first qubit
        qasm2.h(qreg[0])
        # Create a cascading sequence of CX gates
        # necessary for quantum computers that
        # only have nearest-neighbor connectivity between qubits
        for i in range(1, n_qubits):
            qasm2.cx(qreg[i - 1], qreg[i])

    return ghz_linear_program

In [11]:
# --- Somewhere else in your code, ensure ghz_linear is defined and decorated ---
# Example structure:
#
# from bloqade import qasm2
# import math # if needed
#
# @qasm2.extended # This decorator includes core and other common dialects
# def ghz_linear(n_qubits: int):
#     q = qasm2.qreg(n_qubits)
#     qasm2.h(q[0])
#     for i in range(n_qubits - 1):
#         qasm2.cx(q[i], q[i+1])
#     qasm2.measure_all() # Example measurement
#     return q # Return the register

# --- Your original code, corrected ---
from bloqade.qasm2.emit import QASM2
from bloqade.qasm2.parse import pprint # Often useful to print the result

# Import the decorated function if it's in another file
# from your_module import ghz_linear

# Initialize the QASM2 emitter (without 'dialects')
# Add allow_parallel=True if ghz_linear uses the parallel dialect features
target = QASM2()
# target = QASM2(allow_parallel=True) # <--- Uncomment if using parallel gates

# Emit the AST by calling the decorated function
# Make sure ghz_linear is defined and decorated as above
try:
    ast = target.emit(ghz_linear(2)) # Pass the result of calling the decorated function

    # Print the generated QASM AST (optional)
    print("Generated QASM AST:")
    pprint(ast)

except NameError:
    print("Error: The function 'ghz_linear' seems to be undefined.")
    print("Please ensure ghz_linear(n) is defined and decorated with @qasm2.extended.")
except Exception as e:
    print(f"An error occurred during emission: {e}")

Generated QASM AST:
OPENQASM 2.0;
include "qelib1.inc";
qreg qreg[4];
h qreg[0];
CX qreg[0], qreg[1];
CX qreg[1], qreg[2];
CX qreg[2], qreg[3];


The webpage for source code: https://github.com/QuEraComputing/bloqade-pyqrack/blob/main/src/bloqade/pyqrack/qasm2/parallel.py



In [5]:
from typing import Any

from bloqade.qasm2.dialects import parallel


In [6]:
import kirin.interp



@qasm2.extended
def global_H():
    qreg = qasm2.qreg(3)
    
    qasm2.parallel.u(
        qreg,          # Pass the list of qubits
        theta=math.pi/2, # Specify the theta angle
        phi=0,           # Specify the phi angle
        lam=math.pi      # Specify the lambda angle
    )
    qasm2.zz(qreg[0])
    qasm2.h(qreg[1])
    qasm2.zz(qreg[1])
    qasm2.h(qreg[2])
    return global_H


target = QASM2()
ast = target.emit(ghz_linear(2))
pprint(ast)


OPENQASM 2.0;
include "qelib1.inc";
qreg qreg[4];
h qreg[0];
CX qreg[0], qreg[1];
CX qreg[1], qreg[2];
CX qreg[2], qreg[3];
